# PageRank

## Обработка данных

In [1]:
%pip install scipy
%pip install numpy
%pip install pandas
%pip install networkx
%pip install matplotlib

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from typing import List, Dict, Any
from typing import Mapping, Sequence
from itertools import combinations

In [3]:
df = pd.read_csv("res\\rating.csv")

In [4]:
print(df['rating'].describe())

count    2.000026e+07
mean     3.525529e+00
std      1.051989e+00
min      5.000000e-01
25%      3.000000e+00
50%      3.500000e+00
75%      4.000000e+00
max      5.000000e+00
Name: rating, dtype: float64


In [5]:
def get_graph(data: Sequence[Mapping[str, Any]], score_column: str, user_column: str,
              target_column: str) -> nx.DiGraph:
    df = pd.DataFrame(data)

    # Создание пустого графа
    G = nx.DiGraph()

    # MAX_SCORE - максимальная оценка
    MAX_SCORE = df[score_column].max()

    # Группировка данных по пользователям
    grouped = df.groupby(user_column)

    # Проход по каждому пользователю
    for _, movies in grouped:
        movies_rating = movies[score_column].values
        diff_arr = np.subtract.outer(movies_rating, movies_rating) + MAX_SCORE + 1
        source = movies[target_column].values
        source_len = len(source)
        for i in range(source_len):
            for j in range(source_len):
                if i != j:  # Исключаем петли (i, i)
                    weight = diff_arr[j][i]
                    if not G.has_edge(source[i], source[j]):
                        G.add_edge(source[i], source[j], weight=weight)
                    else:
                        G[source[i]][source[j]]['weight'] += weight
    
    return G

In [6]:
# тест метода
data = {
    'userId': [1, 1, 1, 2, 2, 2],
    'movieId': [101, 102, 103, 201, 202, 203],
    'rating': [4, 3, 5, 2, 4, 3]
}
test_graph = get_graph(data=data, score_column='rating', user_column='userId',
          target_column='movieId')

# Вывод ребер графа вместе с весами
for u, v, attrs in test_graph.edges(data=True):
    print(f"Ребро между узлами {u} и {v} имеет вес {attrs['weight']}")

Ребро между узлами 101 и 102 имеет вес 5
Ребро между узлами 101 и 103 имеет вес 7
Ребро между узлами 102 и 101 имеет вес 7
Ребро между узлами 102 и 103 имеет вес 8
Ребро между узлами 103 и 101 имеет вес 5
Ребро между узлами 103 и 102 имеет вес 4
Ребро между узлами 201 и 202 имеет вес 8
Ребро между узлами 201 и 203 имеет вес 7
Ребро между узлами 202 и 201 имеет вес 4
Ребро между узлами 202 и 203 имеет вес 5
Ребро между узлами 203 и 201 имеет вес 5
Ребро между узлами 203 и 202 имеет вес 7


In [7]:
G = get_graph(df.iloc[:50_000], user_column='userId', target_column='movieId', score_column='rating')
# 10 - 8,7s

In [13]:
# Вычисление PageRank

personalization = {}
movies_for_user_1 = df.loc[df['userId'] == 1]['movieId'].tolist()  # Получаем список фильмов для пользователя 1
for movie_id in movies_for_user_1:
    personalization[movie_id] = 1 
    
pagerank = nx.pagerank(G, personalization=personalization)

# Вывод первых 10 рейтингов
top_pagerank = sorted(pagerank.items(), key=lambda x: x[1], reverse=True)[:10]
for node, rank in top_pagerank:
    print(f"Узел: {node}, Рейтинг: {rank}")

Узел: 296, Рейтинг: 0.002488815403569689
Узел: 318, Рейтинг: 0.002364995843628732
Узел: 593, Рейтинг: 0.0023149169390561334
Узел: 50, Рейтинг: 0.0022345430942654035
Узел: 1198, Рейтинг: 0.0022334470988950835
Узел: 1196, Рейтинг: 0.0022264369591044
Узел: 2959, Рейтинг: 0.002209674704521502
Узел: 47, Рейтинг: 0.0021525895028870726
Узел: 589, Рейтинг: 0.0021436707469144065
Узел: 260, Рейтинг: 0.002141750244486403


In [14]:
df_movie = pd.read_csv('res/movie.csv')

In [15]:
movieId_pagerank = pd.DataFrame(list(pagerank.items()), columns=['movieId', 'pagerank'])
merged_df = pd.merge(df_movie, movieId_pagerank, on='movieId', how='inner')
merged_df.head()

,movieId,title,genres,pagerank
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,0.001236
1,2,Jumanji (1995),Adventure|Children|Fantasy,0.001483
2,3,Grumpier Old Men (1995),Comedy|Romance,0.000300
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,0.000108
4,5,Father of the Bride Part II (1995),Comedy,0.000273


In [16]:
merged_df = merged_df.sort_values(by='pagerank', ascending=False)
merged_df.head()

,movieId,title,genres,pagerank
262,296,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,0.002489
281,318,"Shawshank Redemption, The (1994)",Crime|Drama,0.002365
523,593,"Silence of the Lambs, The (1991)",Crime|Horror|Thriller,0.002315
46,50,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,0.002235
939,1198,Raiders of the Lost Ark (Indiana Jones and the...,Action|Adventure,0.002233
